In [2]:
# SUBMIT = True

# if SUBMIT:
#     %pip install crunch-cli --upgrade
#     !crunch setup --notebook causality-discovery default --token Vi5Udk6qYM29z9JMNDuUlgYGRfcqPCD1YUSm07GsIs96fY8JcXfjvOI1i0ve1m8I
#     %pip install crunch-cli --upgrade
#     %pip install gcastle torch
#     pass
# else:
import os
os.chdir('./')

!crunch setup --notebook causality-discovery default --token Vi5Udk6qYM29z9JMNDuUlgYGRfcqPCD1YUSm07GsIs96fY8JcXfjvOI1i0ve1m8I

you appear to have never submitted code before
download data\X_train.pickle from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/X_train.pickle (1523944532 bytes)
download data\y_train.pickle from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/y_train.pickle (7017735 bytes)
download data\X_test.pickle from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/X_test_reduced.pickle (122341879 bytes)
download data\y_test.pickle from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/y_test_reduced.pickle (562930 bytes)
download data\example_prediction.parquet from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/example_prediction_reduced.parquet (668981 bytes)

---
Success! Your environment has been correctly setup.
Next recommended actions:
 - To see all of the available commands of the CrunchDAO CLI, run: crunch


  0%|          | 0.00/1.52G [00:00<?, ?iB/s]
  0%|          | 32.8k/1.52G [00:00<1:40:40, 252kiB/s]
  0%|          | 81.9k/1.52G [00:00<1:23:24, 304kiB/s]
  0%|          | 180k/1.52G [00:00<53:09, 478kiB/s]   
  0%|          | 410k/1.52G [00:00<27:59, 907kiB/s]
  0%|          | 672k/1.52G [00:00<20:43, 1.23MiB/s]
  0%|          | 1.38M/1.52G [00:00<10:28, 2.42MiB/s]
  0%|          | 2.79M/1.52G [00:01<05:21, 4.73MiB/s]
  0%|          | 5.60M/1.52G [00:01<02:44, 9.21MiB/s]
  1%|          | 9.40M/1.52G [00:01<01:52, 13.5MiB/s]
  1%|          | 13.5M/1.52G [00:01<01:28, 17.1MiB/s]
  1%|          | 16.7M/1.52G [00:01<01:24, 17.9MiB/s]
  1%|▏         | 20.8M/1.52G [00:01<01:22, 18.2MiB/s]
  2%|▏         | 24.8M/1.52G [00:02<01:21, 18.5MiB/s]
  2%|▏         | 28.7M/1.52G [00:02<01:18, 19.0MiB/s]
  2%|▏         | 32.8M/1.52G [00:02<01:13, 20.4MiB/s]
  2%|▏         | 36.7M/1.52G [00:02<01:13, 20.3MiB/s]
  3%|▎         | 40.8M/1.52G [00:02<01:08, 21.5MiB/s]
  3%|▎         | 44.1M/1.52G [00:02<

In [3]:
"""
This is a basic example of what you need to do to enter the competition.
The code will not have access to the internet (or any socket related operation).
"""

import os
import typing

import castle.algorithms
import joblib
import networkx as nx
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from utils import *
# keep me, I am needed by castle
import torch

2024-09-15 13:41:45,400 - e:\Softwares\Anaconda\CondaEnvs\casual\lib\site-packages\castle\backend\__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2024-09-15 13:41:45,623 - e:\Softwares\Anaconda\CondaEnvs\casual\lib\site-packages\castle\algorithms\__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.
e:\Softwares\Anaconda\CondaEnvs\casual\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import crunch
crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>


In [5]:
def fix_DAG(g):
    """
    Ensure that the graph is a DAG and has an edge X→Y

    We look for cycles, and remove an edge in each cycle, until there are no cycles left.

    Inputs: g: nx.DiGraph
    Output: g: nx.DiGraph

    This function provides just a possible solution to the problem
    of DAG-ifying a graph. Other solutions can be conceived that could
    be better for the competition.
    """

    assert 'X' in g.nodes
    assert 'Y' in g.nodes

    gg = g.copy()

    # Add X→Y if it is missing
    if ('X', 'Y') not in gg.edges:
        gg.add_edge('X', 'Y')

    # Look for cycles and remove them
    while not nx.is_directed_acyclic_graph(gg):

        h = gg.copy()

        # Remove all the sources and sinks
        while True:
            finished = True

            for i, v in nx.in_degree_centrality(h).items():
                if v == 0:
                    h.remove_node(i)
                    finished = False

            for i, v in nx.out_degree_centrality(h).items():
                if v == 0:
                    h.remove_node(i)
                    finished = False

            if finished:
                break

        # Find a cycle, with a random walk starting at a random node
        node = list(h.nodes)[0]
        cycle = [node]
        while True:
            edges = list(h.out_edges(node))
            _, node = edges[np.random.choice(len(edges))]

            if node in cycle:
                break

            cycle.append(node)

        # We have a path that ends with a cycle: remove the begining, if it is not part of the cycle
        cycle = np.array(cycle)
        i = np.argwhere(cycle == node)[0][0]
        cycle = cycle[i:]
        cycle = cycle.tolist() + [node]

        # Edges in that cycle
        edges = list(zip(cycle[:-1], cycle[1:]))

        # Pick an edge at random, but make sure it is not X→Y -- we want to keep that one
        edges = [e for e in edges if e != ('X', 'Y')]
        edge = edges[np.random.choice(len(edges))]

        gg.remove_edge(*edge)

    return gg

In [6]:
# Uncomment what you need!
def train(
    X_train: typing.Dict[str, pd.DataFrame],
    y_train: typing.Dict[str, pd.DataFrame],
    # number_of_features: int,
    model_directory_path: str,
    # id_column_name: str,
    # prediction_column_name: str,
    # has_gpu: bool,
) -> None:
    # TODO replace me with a real model
    model = ...

    joblib.dump(
        model,
        os.path.join(model_directory_path, "model.joblib")
    )

In [7]:
# Uncomment what you need!
def infer(
    X_test: typing.Dict[str, pd.DataFrame],
    # number_of_features: int,
    model_directory_path: str,
    id_column_name: str,
    prediction_column_name: str,
    # has_gpu: bool,
    # has_trained: bool,
) -> pd.DataFrame:
    # TODO use me
    # model = joblib.load(os.path.join(model_directory_path, "model.joblib"))

    predictions = {}
    for dataset_id in tqdm(X_test):
        X = X_test[dataset_id]

        nodes = X.columns
        model = castle.algorithms.PC()
        model.learn(X)

        A_hat = pd.DataFrame(model.causal_matrix, columns=nodes, index=nodes)
        g_hat = nx.from_pandas_adjacency(A_hat, create_using=nx.DiGraph)
        g_hat = fix_DAG(g_hat)

        G = pd.DataFrame(nx.to_numpy_array(g_hat).astype(int), columns=nodes, index=nodes)
        for i in nodes:
            for j in nodes:
                predictions[f'{dataset_id}_{i}_{j}'] = int(G.loc[i, j])

    return pd.DataFrame(
        predictions.items(),
        columns=[id_column_name, prediction_column_name]
    )

In [8]:
crunch.test(
    no_determinism_check=True
)

print("Download this notebook and submit it to the platform: https://hub.crunchdao.com/competitions/causality-discovery/submit/via/notebook")  

13:48:07 no forbidden library found
13:48:07 
13:48:07 started
13:48:07 running local test
13:48:07 internet access isn't restricted, no check will be done
13:48:07 
13:48:14 starting dag process...


download data\X_train.pickle from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/X_train.pickle (1523944532 bytes)
already exists: file length match
download data\y_train.pickle from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/y_train.pickle (7017735 bytes)
already exists: file length match
download data\X_test.pickle from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/X_test_reduced.pickle (122341879 bytes)
already exists: file length match
download data\y_test.pickle from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/y_test_reduced.pickle (562930 bytes)
already exists: file length match
download data\example_prediction.parquet from https:crunchdao--competition--production.s3.eu-west-1.amazonaws.com/data-releases/48/example_prediction_reduced.parquet (668981 bytes)
already exists: file length match


13:48:25 call: train
13:48:25 call: infer
  3%|▎         | 52/1880 [00:04<02:09, 14.13it/s]e:\Softwares\Anaconda\CondaEnvs\casual\lib\site-packages\castle\common\independence_tests.py:86: RuntimeWarning: invalid value encountered in scalar divide
  r = -1 * PM[0, 1] / math.sqrt(abs(PM[0, 0] * PM[1, 1]))
  7%|▋         | 127/1880 [00:10<01:35, 18.28it/s]e:\Softwares\Anaconda\CondaEnvs\casual\lib\site-packages\castle\common\independence_tests.py:86: RuntimeWarning: divide by zero encountered in scalar divide
  r = -1 * PM[0, 1] / math.sqrt(abs(PM[0, 0] * PM[1, 1]))
100%|██████████| 1880/1880 [02:27<00:00, 12.75it/s]
13:50:54 save prediction - path=data\prediction.csv
13:50:55 check prediction - call=columns_name({})
13:50:55 check prediction - call=nans({})
13:50:55 check prediction - call=ids({})
13:50:55 check prediction - call=values_allowed({'values': [0, 1]}) column=`prediction`
13:50:55 prediction is valid
13:50:55 ended
13:50:55 duration - time=00:02:48
13:50:55 memory - before="3

Download this notebook and submit it to the platform: https://hub.crunchdao.com/competitions/causality-discovery/submit/via/notebook
